In [1]:
import requests
import json
from datetime import date
from dateutil.rrule import rrule, DAILY

iss_url_raw = 'https://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json'
start_date = date(2023, 5, 25)
end_date = start_date #date(2023, 5, 8)

file_name = 'price_collection.json'
with open(file_name, "r") as read_file:
    tickers_list = json.load(read_file)
    read_file.close()

for d in rrule(DAILY, dtstart=start_date, until=end_date):
    iss_date = d.strftime("%Y-%m-%d")
    iss_start = 0
    tickers_count = 1
    while iss_start < tickers_count:
        iss_url = iss_url_raw + '?date=' + iss_date + '&start=' + str(iss_start)
        response = requests.get(iss_url)
        raw_data = json.loads(response.text)
        if iss_start == 0:
            tickers_count = raw_data["history.cursor"]["data"][0][1]
            columns_raw = raw_data["history"]["columns"]
            column_tradedate = columns_raw.index('TRADEDATE')
            column_secid = columns_raw.index('SECID')
            column_open = columns_raw.index('OPEN')
            column_low = columns_raw.index('LOW')
            column_high = columns_raw.index('HIGH')
            column_close = columns_raw.index('CLOSE')
            column_volume = columns_raw.index('VOLUME')
        if tickers_count > 0:
            raw_data = raw_data["history"]["data"]  # list of ticker lists
            for ticker in raw_data:
                if ticker[column_volume] == 0:
                    one_ticker = [ticker[column_tradedate], ticker[column_secid], ticker[column_volume],
                                  0, 0, 0, 0]
                else:
                    one_ticker = [ticker[column_tradedate], ticker[column_secid], ticker[column_volume],
                                  ticker[column_open], ticker[column_low], ticker[column_high], ticker[column_close]]
                tickers_list.append(one_ticker)
            iss_start += 100
        else:
            tickers_count = -1

with open(file_name, "w", encoding='utf-8') as write_file:
    json.dump(tickers_list, write_file, indent=4, separators=(',', ':'))
    write_file.close()